In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
from matplotlib.ticker import NullFormatter
import sys
import scipy
from __future__ import print_function, division

import os
import math
import time

from matplotlib import rcParams
from matplotlib.ticker import MultipleLocator, AutoLocator, AutoMinorLocator
from matplotlib import gridspec
from matplotlib import cm

In [2]:
import xpsi

/=============================================\
| X-PSI: X-ray Pulse Simulation and Inference |
|---------------------------------------------|
|                Version: 1.2.1               |
|---------------------------------------------|
|      https://xpsi-group.github.io/xpsi      |
\=============================================/

Imported GetDist version: 0.3.1
Imported nestcheck version: 0.2.0


In [3]:
from xpsi.global_imports import _c, _G, _dpr, gravradius, _csq, _km, _2pi

In [4]:
bounds = dict(distance = (0.1, 1.0),                     # (Earth) distance
                mass = (1.0, 3.0),                       # mass
                radius = (3.0 * gravradius(1.0), 16.0),  # equatorial radius
                cos_inclination = (0.0, 1.0))      # (Earth) inclination to rotation axis

spacetime = xpsi.Spacetime(bounds=bounds, values=dict(frequency=572.0))

Creating parameter:
    > Named "frequency" with fixed value 5.720e+02.
    > Spin frequency [Hz].
Creating parameter:
    > Named "mass" with bounds [1.000e+00, 3.000e+00].
    > Gravitational mass [solar masses].
Creating parameter:
    > Named "radius" with bounds [4.430e+00, 1.600e+01].
    > Coordinate equatorial radius [km].
Creating parameter:
    > Named "distance" with bounds [1.000e-01, 1.000e+00].
    > Earth distance [kpc].
Creating parameter:
    > Named "cos_inclination" with bounds [0.000e+00, 1.000e+00].
    > Cosine of Earth inclination to rotation axis.


In [5]:
filename = '/home/pushpita/Documents/codes_dir/3D_final_runs/inc60/data_r+0.420D+01_n0508.csv'

In [6]:
bounds = dict(super_colatitude = (None, None),
              super_radius = (None, None),
              phase_shift = (0.0, 0.1),
              super_temperature = (None, None))

primary = xpsi.HotRegion(bounds=bounds,
	                    values={},
	                    symmetry=True,
	                    omit=False,
	                    cede=False,
	                    concentric=False,
	                    sqrt_num_cells=256,
	                    min_sqrt_num_cells=128,
	                    max_sqrt_num_cells=512,
	                    num_leaves=256,
	                    num_rays=400,
	                    prefix='p',
						mycoolgrid = True,
						first_spot = True,
                        filename=filename)

Creating parameter:
    > Named "super_colatitude" with bounds [0.000e+00, 3.142e+00].
    > The colatitude of the centre of the superseding region [radians].
Creating parameter:
    > Named "super_radius" with bounds [0.000e+00, 1.571e+00].
    > The angular radius of the (circular) superseding region [radians].
Creating parameter:
    > Named "phase_shift" with bounds [0.000e+00, 1.000e-01].
    > The phase of the hot region, a periodic parameter [cycles].
Creating parameter:
    > Named "super_temperature" with bounds [3.000e+00, 7.600e+00].
    > log10(superseding region effective temperature [K]).


In [7]:
secondary = xpsi.HotRegion(bounds=bounds,
                              values={}, # no fixed/derived variables
                              symmetry=True,
                              omit=False,
                              cede=False,
                              concentric=False,
                              sqrt_num_cells=256,
                              min_sqrt_num_cells=128,
                              max_sqrt_num_cells=512,
                              num_leaves=256,
                              num_rays=400,
                              do_fast=False,
                              is_secondary=False,
                              is_antiphased=True,
                              prefix='s',
                              mycoolgrid = True,
                              second_spot = True,
                              filename = filename)

from xpsi import HotRegions

hot = HotRegions((primary, secondary))

Creating parameter:
    > Named "super_colatitude" with bounds [0.000e+00, 3.142e+00].
    > The colatitude of the centre of the superseding region [radians].
Creating parameter:
    > Named "super_radius" with bounds [0.000e+00, 1.571e+00].
    > The angular radius of the (circular) superseding region [radians].
Creating parameter:
    > Named "phase_shift" with bounds [0.000e+00, 1.000e-01].
    > The phase of the hot region, a periodic parameter [cycles].
Creating parameter:
    > Named "super_temperature" with bounds [3.000e+00, 7.600e+00].
    > log10(superseding region effective temperature [K]).


In [8]:
elsewhere = xpsi.Elsewhere(sqrt_num_cells=16,bounds=dict(elsewhere_temperature = (None,None)),
                           mycoolgrid=False,
                           myelsewhere=True,
                           filename=filename)

Creating parameter:
    > Named "elsewhere_temperature" with bounds [3.000e+00, 7.600e+00].
    > log10 of the effective temperature elsewhere.


In [9]:
photosphere = xpsi.Photosphere(hot = hot, elsewhere = elsewhere,
                               values=dict(mode_frequency = spacetime['frequency']))
star = xpsi.Star(spacetime = spacetime, photospheres = photosphere)
star

Creating parameter:
    > Named "mode_frequency" with fixed value 5.720e+02.
    > Coordinate frequency of the mode of radiative asymmetry in the
photosphere that is assumed to generate the pulsed signal [Hz].


Free parameters
---------------
mass: Gravitational mass [solar masses].
radius: Coordinate equatorial radius [km].
distance: Earth distance [kpc].
cos_inclination: Cosine of Earth inclination to rotation axis.
p__phase_shift: The phase of the hot region, a periodic parameter [cycles].
p__super_colatitude: The colatitude of the centre of the superseding region [radians].
p__super_radius: The angular radius of the (circular) superseding region [radians].
p__super_temperature: log10(superseding region effective temperature [K]).
s__phase_shift: The phase of the hot region, a periodic parameter [cycles].
s__super_colatitude: The colatitude of the centre of the superseding region [radians].
s__super_radius: The angular radius of the (circular) superseding region [radians].
s__super_temperature: log10(superseding region effective temperature [K]).
elsewhere_temperature: log10 of the effective temperature elsewhere.

In [10]:
p = [1.68,
     10.5,
     0.2,
     math.cos(2.5),
     0.0,
     0.0,
     math.pi/2 - 0.05,
     5.0,
     0.0,
     math.pi,
     math.pi/2-0.05,
     5.0,
     5.0]

star(p)
star.params

[Gravitational mass [solar masses] = 1.680e+00,
 Coordinate equatorial radius [km] = 1.050e+01,
 Earth distance [kpc] = 2.000e-01,
 Cosine of Earth inclination to rotation axis = -8.011e-01,
 The phase of the hot region, a periodic parameter [cycles] = 0.000e+00,
 The colatitude of the centre of the superseding region [radians] = 0.000e+00,
 The angular radius of the (circular) superseding region [radians] = 1.521e+00,
 log10(superseding region effective temperature [K]) = 5.000e+00,
 The phase of the hot region, a periodic parameter [cycles] = 0.000e+00,
 The colatitude of the centre of the superseding region [radians] = 3.142e+00,
 The angular radius of the (circular) superseding region [radians] = 1.521e+00,
 log10(superseding region effective temperature [K]) = 5.000e+00,
 log10 of the effective temperature elsewhere = 5.000e+00]

In [11]:
#star['cos_inclination'] = math.cos(50.*np.pi/180.)
#star.update(force_update=True)
#d = photosphere.embed
energies = np.logspace(-2.0, np.log10(5.0), 50, base=10.0)
photosphere.integrate(energies, threads=1) # the number of OpenMP threads to use

AttributeError: 'Elsewhere' object has no attribute '_cellArea'

In [ ]:
rcParams['text.usetex'] = False
rcParams['font.size'] = 14.0

def veneer(x, y, axes, lw=1.0, length=8, yticks=None):
    """ Make the plots a little more aesthetically pleasing. """
    if x is not None:
        if x[1] is not None:
            axes.xaxis.set_major_locator(MultipleLocator(x[1]))
        if x[0] is not None:
            axes.xaxis.set_minor_locator(MultipleLocator(x[0]))
    else:
        axes.xaxis.set_major_locator(AutoLocator())
        axes.xaxis.set_minor_locator(AutoMinorLocator())

    if y is not None:
        if y[1] is not None:
            axes.yaxis.set_major_locator(MultipleLocator(y[1]))
        if y[0] is not None:
            axes.yaxis.set_minor_locator(MultipleLocator(y[0]))
    else:
        axes.yaxis.set_major_locator(AutoLocator())
        axes.yaxis.set_minor_locator(AutoMinorLocator())

    axes.tick_params(which='major', colors='black', length=length, width=lw)
    axes.tick_params(which='minor', colors='black', length=int(length/2), width=lw)
    plt.setp(axes.spines.values(), linewidth=lw, color='black')

    if yticks:
        axes.set_yticks(yticks)

def plot_pulse():
    """ Plot hot region signals before telescope operation. """
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(111)

    ax.set_ylabel('Signal [arbitrary normalisation]')
    ax.set_xlabel('Phase [cycles]')

    temp = np.sum(photosphere.signal[0][0], axis=0)
    temp1 = np.sum(photosphere.signal[0][0], axis=0)
    #ax.plot(hot.phases_in_cycles[0], (temp)/np.max(temp), 'o-', color='k', lw=0.5, markersize=2)
    ax.plot(hot.phases_in_cycles[0], photosphere.signal[0][0][1]/np.max(photosphere.signal[0][0][1]), 'o-', color='r', lw=0.5, markersize=2)
    print(temp1+temp)
    #veneer((0.05,0.2), (0.05,0.2), ax)
_ = plot_pulse()

In [ ]:
from xpsi.tools import phase_interpolator

In [ ]:
def plot_2D_pulse(z, x, shift, y, ylabel,
                  num_rotations=5.0, res=5000, cm=cm.viridis,
                  yticks=None):
    """ Helper function to plot a phase-energy pulse.

    :param array-like z:
        A pair of *ndarray[m,n]* objects representing the signal at
        *n* phases and *m* values of an energy variable.

    :param ndarray[n] x: Phases the signal is resolved at.

    :param tuple shift: Hot region phase parameters.

    :param ndarray[m] x: Energy values the signal is resolved at.

    """

    fig = plt.figure(figsize = (12,6))

    gs = gridspec.GridSpec(1, 2, width_ratios=[50,1], wspace=0.025)
    ax = plt.subplot(gs[0])
    ax_cb = plt.subplot(gs[1])

    new_phases = np.linspace(0.0, num_rotations, res)

    interpolated = phase_interpolator(new_phases,
                                      x,
                                      z[0], shift[0])
    interpolated += phase_interpolator(new_phases,
                                       x,
                                       z[1], shift[1])

    profile = ax.pcolormesh(new_phases,
                             y,
                             interpolated/np.max(interpolated),
                             cmap = cm,
                             linewidth = 0,
                             rasterized = True)

    profile.set_edgecolor('face')

    ax.set_xlim([0.0, num_rotations])
    ax.set_yscale('log')
    ax.set_ylabel(ylabel)
    ax.set_xlabel(r'Phase')
    veneer((0.1, 0.5), (None,None), ax, yticks=yticks)
    if yticks is not None:
        ax.set_yticklabels(yticks)

    cb = plt.colorbar(profile,
                      cax = ax_cb,
                      ticks = MultipleLocator(0.2))

    cb.set_label(label=r'Signal (normalised by maximum)', labelpad=25)
    cb.solids.set_edgecolor('face')

    veneer((None, None), (0.05, None), ax_cb)
    cb.outline.set_linewidth(1.0)

In [ ]:
plot_2D_pulse((photosphere.signal[0][0], photosphere.signal[1][0]),
              x=hot.phases_in_cycles[0],
              shift=[ hot['p__phase_shift'], hot['s__phase_shift'] ],
              y=energies,
              ylabel=r'Energy (keV)',
              yticks=[0.1,0.3,1.0,3.0])

In [ ]:
print(photosphere.signal[0][0][0],np.shape(photosphere.signal),np.shape(hot.phases_in_cycles[0]))

In [ ]:
fig = plt.figure(figsize = (100,50))

gs = gridspec.GridSpec(1, 3, width_ratios=[50,50,1], wspace=0.2)
ax = plt.subplot(gs[0])
veneer((1,5), (1, 5), ax)

# primary (lower colatitude) hot region
h = hot.objects[0]
z = h._HotRegion__cellArea[0]/np.max(h._HotRegion__cellArea[0])
patches = plt.pcolormesh(z,
                         vmin = np.min(z),
                         vmax = np.max(z),
                         cmap = cm.magma,
                         linewidth = 1.0,
                         rasterized = True,
                         edgecolor='black')

ax = plt.subplot(gs[1])
veneer((1,5), (1, 5), ax)

# secondary (higher colatitude) hot region
h = hot.objects[1]
z = h._HotRegion__cellArea[0]/np.max(h._HotRegion__cellArea[0])
_ = plt.pcolormesh(z,
                   vmin = np.min(z),
                   vmax = np.max(z),
                   cmap = cm.magma,
                   linewidth = 1.0,
                   rasterized = True,
                   edgecolor='black')

ax_cb = plt.subplot(gs[2])
cb = plt.colorbar(patches,
                  cax = ax_cb,
                  ticks = MultipleLocator(0.2))

cb.set_label(label = r'cell area (normalised by maximum)', labelpad=25)
cb.solids.set_edgecolor('face')

veneer((None, None), (0.05, None), ax_cb)
cb.outline.set_linewidth(1.0)

In [ ]:
fig = plt.figure(figsize = (20,7))

gs = gridspec.GridSpec(1, 3, width_ratios=[50,50,1], wspace=0.2)
ax = plt.subplot(gs[0])

# primary (lower colatitude) hot region
h = hot.objects[0]
z = h._super_cellParamVecs[:,:,0]#/np.max(h._HotRegion__cellArea[0])
patches = plt.pcolormesh(h._super_phi,
                        h._super_theta,
                        z,
                        vmin = np.min(z),
                        vmax = np.max(z),
                        cmap = cm.magma,
                        linewidth = 1.0,
                        rasterized = True)

ax = plt.subplot(gs[1])

# secondary (higher colatitude) hot region
h = hot.objects[1]
z = h._super_cellParamVecs[:,:,0]#_HotRegion__cellArea[0]/np.max(h._HotRegion__cellArea[0])
_ = plt.pcolormesh(h._super_phi,
                    h._super_theta,
                    z,
                    vmin = np.min(z),
                    vmax = np.max(z),
                    cmap = cm.magma,
                    linewidth = 1.0,
                    rasterized = True)

ax_cb = plt.subplot(gs[2])
cb = plt.colorbar(patches,
                  cax = ax_cb,
                  ticks = MultipleLocator(0.2))

cb.set_label(label = r'Temperatures', labelpad=25)
cb.solids.set_edgecolor('face')

veneer((None, None), (0.05, None), ax_cb)
cb.outline.set_linewidth(1.0)


In [ ]:
spacetime.a = 0.0 # spacetime spin parameter (~angular momentum)
spacetime.q = 0.0 # spacetime mass quadrupole moment

sky_map_kwargs = {'panel_indices': (0,1,2,3,4,5), # select energy indexes
                  'num_levels': 500,
                  'colormap': cm.Purples_r,
                  'phase_average': True,
                  'annotate_energies': True,
                  'energy_annotation_format': '[%.2f keV]',
                  'annotate_location': (0.025,0.025)} # do not phase average if you want to animate a sequence

In [ ]:
animate_kwargs = {'cycles': 8, 'fps': 32}

photosphere.image(reimage = True,
                  cache_intensities = 1.0, # cache size limit in GBs
                  energies = np.array([0.01,0.1,0.5,1.0,2.0,5.0]),
                  phases = hot.phases_in_cycles[0] * _2pi,
                  sqrt_num_rays = 400,
                  threads = 4,
                  max_steps = 100000,
                  epsrel_ray = 1.0e-12,
                  plot_sky_maps = True, # activate if you want to plot frames
                  sky_map_kwargs = sky_map_kwargs,
                  animate_sky_maps = True, # activate if you want to animate
                  free_memory = True, # try to free ray-map/intensity cache memory before animating
                  animate_kwargs = animate_kwargs)

In [ ]:
import os
import numpy as np
import math

from matplotlib import pyplot as plt
import xpsi
from xpsi.utilities.ProjectionTool import plot_projection_general

In [ ]:
# EXAMPLE: ST+PDT (primary composed of a single temperature component and
#          secondary of a double temperature hot spot with protruding superseding component)
# setting the dictionary including the necessary parameters
# EXAMPLE: ST-U, setting parameter values
labels = ['mass',
          'radius',
          'cos_inclination',
          's__phase_shift',
          's__super_colatitude',
          's__super_radius',
          's__super_temperature',
          'p__phase_shift',
          'p__super_colatitude',
          'p__super_radius',
          'p__super_temperature',
          'beta',
          'column_density']

values_2hsSTU = [ 1.4, 15,  0.1,  0,  math.pi,
        math.pi/2 - 0.01,  6.15,  0,  0, math.pi/2 - 0.01,
        6.1, 9.3, 0.8]

P_dictionary_2hsSTU =  dict(zip(labels, values_2hsSTU))

P_dictionary_2hsSTU

In [ ]:
# from Earth and phase 0.
plot_projection_general((P_dictionary_2hsSTU),"ST+ST","I","NP")


In [ ]:
import numpy as np

theta = np.linspace(0,np.pi,5)
phi = np.linspace(-np.pi,np.pi,5)
ph,th = np.meshgrid(phi,theta)

d = np.ones((5,5,2))
d[:,:,0] = th[:,:]
print(d[:,:,:-1])
print(d[:,:,-1])

In [ ]:
xd1 = (xloc(1)-x(ic11,ic12,1)) / (x(ic21,ic12,1) - x(ic11,ic12,1))

xd2 = (xloc(2)-x(ic11,ic12,2)) / (x(ic11,ic22,2) - x(ic11,ic12,2))

c00 = gf(ic11,ic12) * (1.0d0 - xd1) + gf(ic21,ic12) * xd1
c10 = gf(ic11,ic22) * (1.0d0 - xd1) + gf(ic21,ic22) * xd1

gfloc  = c00 * (1.0d0 - xd2) + c10 * xd2
